In [2]:
import camelot
import os
import pandas as pd
import glob
import re


In [65]:

def clean_page_1(df):
    
    #Step 1: Prepare the DataFrame
    #Drop the first column containing the paragraph and renaming the second column
    # df = df.drop(columns=[0])
    # df.columns = ['Raw'] + list(df.columns[1:])
    # print(df)
    
    #Step 2: Manually define the paramaters with units
    parameters = [
        "Speed (m/s)",
        "Cadence (steps/min)",
        "Steps",
        "Duration (s)",
        "Distance (m)"
    ]
    values = []
    
    #Step 3: Use re to extract all numeric value and store in a list in a sequential order
    for index, row in df.iterrows():
        text = str(row[1]).strip()
        # print(text)
    
        nums = re.findall(r'\d+\.\d+|\d+', text)
        if len(nums) > 0:
            values.append(nums[0])
            
    
    print(f"Total number of Extracted Values: {values}")
    
    
    
        # Step 4: Creates the new dataframe with 'parameters' and 'values' list
    
    df1 = pd.DataFrame({
        'Parameter Name (Unit)': parameters,
        'Value': values
    })
    
    print("\nPage 1 cleaning is complete")
    
    return df1

    

def clean_page_3(df):
    parameters_3 = [
    "Step Length - Left (cm)",
    "Step Length SD - Left (cm)",
    "Step Length - Right (cm)",
    "Step Length SD - Right (cm)",
    "Step Width - Left (cm)",
    "Step Width SD - Left (cm)",
    "Step Width - Right (cm)",
    "Step Width SD - Right (cm)",
    "Stride Length - Left (cm)",
    "Stride Length SD - Left (cm)",
    "Stride Length - Right (cm)",
    "Stride Length SD - Right (cm)"
    ]
    values_3 = []
    parameter_number = 3
    
    i = 0 #counter to count through each row 
    j = 1
    
    for i in range(parameter_number):
        print(f'in this iteration, j ={j} and i ={i}')
        left_value, left_sd = df.iloc[2, j].split(' ± ')
        right_value,right_sd = df.iloc[3, j].split(' ± ')
                    
        # Append rows in the desired format
        values_3.append(left_value)
        values_3.append(left_sd)
        values_3.append(right_value)
        values_3.append(right_sd)
        
        j+=2
    df3 = pd.DataFrame(list(zip(parameters_3, values_3)))
    print(df3)
    
    
        
        
        
                    

            #         print('hello')
            #         # Append rows in a fixed order to maintain alignment
            #         new_rows.append(['Step Length (Left)', left_value.strip(), 'Value'])
            #         new_rows.append(['Step Length (Left)', left_sd.strip(), 'SD'])
            #         new_rows.append(['Step Length (Right)', right_value.strip(), 'Value'])
            #         new_rows.append(['Step Length (Right)', right_sd.strip(), 'SD'])
                    
            # except ValueError as ve:
            #     print(f"Error in row {i}: {ve}. Skipping this row")









page_cleaning_switch = {
    1: clean_page_1,
    3: clean_page_3
}

def get_csv_name(pdf_filename):
    """
    Extracts the part of the PDF filename before the first '-' and appends '.csv'.
    Example: 'report1-2023.pdf' -> 'report1.csv'
    """
    base_name = os.path.basename(pdf_filename)
    name_part = base_name.split('-')[0]
    csv_name = f"{name_part}.csv"
    return csv_name

# -----------------------------
# 2. Main Extraction Function
# -----------------------------

def extract_data_from_pdfs(pdf_dir, pages, output_dir):
    """
    Extracts data from specified pages in all gait reports within the given directory.
    
    Parameters:
    - pdf_dir (str): Directory containing PDF files.
    - pages (str): Pages to extract data from (e.g., '1,2,3').
    
    Returns:
    - data_frames (list): List of cleaned DataFrames from all PDFs.
    """
    # Find all PDF files in the directory
    pdf_files = glob.glob(os.path.join(pdf_dir, '*.pdf'))

    if not pdf_files:
        print("No PDF files found in the specified directory.")
        return

    # Process each PDF file
    for pdf_file in pdf_files:
        print(f"\nProcessing file: {pdf_file}")
        
        try:
            # Extract tables from specified pages
            tables = camelot.read_pdf(
                pdf_file,
                pages=pages,
                flavor='stream',  # Use 'stream' or 'lattice' depending on your PDFs
                strip_text='\n',  # Remove line breaks within cells
                edge_tol=500,     # Tolerance for table edge detection; adjust as needed
                row_tol=10,       # Tolerance for row detection; adjust as needed
            )

            if tables.n == 0:
                print(f"    No tables found in {pdf_file} on pages {pages}.")
                continue

            print(f"  Found {tables.n} tables in {pdf_file} on pages {pages}.")

            # Iterate over each extracted table
            for i, table in enumerate(tables, start=1):
                # print(table)
                
                df = table.df  # Get the table as a DataFrame
                # print(df)
                # Optional: Clean the DataFrame
                df = df.dropna(how='all')           # Drop rows where all elements are NaN
                df = df.dropna(axis=1, how='all')   # Drop columns where all elements are NaN

                # Get the page number from the table
                page_number = table.page

                # Applying page-specific cleaning using the cleaning functions
                clean_function = page_cleaning_switch.get(page_number)
                if clean_function:
                    df = clean_function(df)
                else:
                    print(f"No cleaning function defined for page {page_number}")
                    continue  # Or handle as appropriate
                
            
            
        except Exception as e:
            print(f"  An error occurred while processing {pdf_file}: {e}")

    

# -----------------------------
# 3. Example Usage
# -----------------------------

pdf_dir = r"C:\Users\yangk\OneDrive\Documents\HATCH\Programming Learning\MotionCloud Reports"
pages_to_extract = '3'
output_dir = r'C:\Users\yangk\OneDrive\Documents\HATCH\Programming Learning\Extracted Data from MotionCloud Report'

# Call the extraction function
extracted_data = extract_data_from_pdfs(pdf_dir, pages_to_extract, output_dir)




            


Processing file: C:\Users\yangk\OneDrive\Documents\HATCH\Programming Learning\MotionCloud Reports\fast_mid1-gait report.pdf
  Found 1 tables in C:\Users\yangk\OneDrive\Documents\HATCH\Programming Learning\MotionCloud Reports\fast_mid1-gait report.pdf on pages 3.
in this iteration, j =1 and i =0
in this iteration, j =3 and i =1
in this iteration, j =5 and i =2
                                0       1
0         Step Length - Left (cm)   84.33
1      Step Length SD - Left (cm)    1.85
2        Step Length - Right (cm)   90.95
3     Step Length SD - Right (cm)    2.08
4          Step Width - Left (cm)   12.32
5       Step Width SD - Left (cm)    1.50
6         Step Width - Right (cm)   12.30
7      Step Width SD - Right (cm)    2.43
8       Stride Length - Left (cm)  175.30
9    Stride Length SD - Left (cm)    2.70
10     Stride Length - Right (cm)  175.48
11  Stride Length SD - Right (cm)    2.93

Processing file: C:\Users\yangk\OneDrive\Documents\HATCH\Programming Learning\MotionCloud 